In [1]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
train_df = pd.read_json("train.json")

In [4]:
train_df.index = train_df["id"]
train_df["inc_angle"] = train_df["inc_angle"].replace('na', 0)
train_df.drop("id",axis=1).head()

,band_1,band_2,inc_angle,is_iceberg
id,,,,
dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [5]:
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_df["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_df["band_2"]])
x_band3 = np.array([np.array([band]*75*75).astype(np.float32).reshape(75, 75) for band in train_df["inc_angle"]])

In [6]:
#x_band3 = (x_band1+x_band2) * x_band3

In [7]:
test_df = pd.read_json("test.json")
test_df.index = test_df["id"]
test_df["inc_angle"] = test_df["inc_angle"].replace('na', 0)
test_df.drop("id",axis=1).head()

x_test_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test_df["band_1"]])
x_test_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test_df["band_2"]])
x_test_band3 = np.array([np.array([band]*75*75).astype(np.float32).reshape(75, 75) for band in test_df["inc_angle"]])

#x_test_band3 = (x_test_band1+x_test_band2) * x_test_band3

In [8]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()

x_band1_std = stdsc.fit_transform(x_band1.flatten()).reshape(-1,75,75)
x_test_band1_std = stdsc.transform(x_test_band1.flatten()).reshape(-1,75,75)
x_band2_std = stdsc.fit_transform(x_band2.flatten()).reshape(-1,75,75)
x_test_band2_std = stdsc.transform(x_test_band2.flatten()).reshape(-1,75,75)
x_band3_std = stdsc.fit_transform(x_band3.flatten()).reshape(-1,75,75)
x_test_band3_std = stdsc.transform(x_test_band3.flatten()).reshape(-1,75,75)

C:\Users\rad-fpga\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\rad-fpga\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\rad-fpga\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a s

In [9]:
X_train = np.concatenate((np.expand_dims(x_band1_std, axis=-1), np.expand_dims(x_band2_std, axis=-1), np.expand_dims(x_band3_std, axis=-1)), axis=-1)
y_train = np.array(train_df["is_iceberg"])
X_test = np.concatenate((np.expand_dims(x_test_band1_std, axis=-1), np.expand_dims(x_test_band2_std, axis=-1), np.expand_dims(x_test_band3_std, axis=-1)), axis=-1)

In [10]:
"""import cv2

def get_more_images(imgs):
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images"""

'import cv2\n\ndef get_more_images(imgs):\n    more_images = []\n    vert_flip_imgs = []\n    hori_flip_imgs = []\n      \n    for i in range(0,imgs.shape[0]):\n        a=imgs[i,:,:,0]\n        b=imgs[i,:,:,1]\n        c=imgs[i,:,:,2]\n        \n        av=cv2.flip(a,1)\n        ah=cv2.flip(a,0)\n        bv=cv2.flip(b,1)\n        bh=cv2.flip(b,0)\n        cv=cv2.flip(c,1)\n        ch=cv2.flip(c,0)\n        \n        vert_flip_imgs.append(np.dstack((av, bv, cv)))\n        hori_flip_imgs.append(np.dstack((ah, bh, ch)))\n      \n    v = np.array(vert_flip_imgs)\n    h = np.array(hori_flip_imgs)\n       \n    more_images = np.concatenate((imgs,v,h))\n    \n    return more_images'

In [11]:
from scipy.ndimage import rotate as rot
def augment(images):
    image_mirror_lr = []
    image_mirror_ud = []
    image_mirror_lrud = []
    image_rotate = []
    image_rotate_2 = []
    image_rotate_3 = []
    image_rotate_4 = []
    for i in range(0,images.shape[0]):
        band_1 = images[i,:,:,0]
        band_2 = images[i,:,:,1]
        band_3 = images[i,:,:,2]
            
        # mirror left-right
        band_1_mirror_lr = np.flip(band_1, 0)
        band_2_mirror_lr = np.flip(band_2, 0)
        band_3_mirror_lr = np.flip(band_3, 0)
        image_mirror_lr.append(np.dstack((band_1_mirror_lr, band_2_mirror_lr, band_3_mirror_lr)))
        
        # mirror up-down
        band_1_mirror_ud = np.flip(band_1, 1)
        band_2_mirror_ud = np.flip(band_2, 1)
        band_3_mirror_ud = np.flip(band_3, 1)
        image_mirror_ud.append(np.dstack((band_1_mirror_ud, band_2_mirror_ud, band_3_mirror_ud)))
        
        band_1_mirror_lrud = np.flip(band_1_mirror_lr, 1)
        band_2_mirror_lrud = np.flip(band_2_mirror_lr, 1)
        band_3_mirror_lrud = np.flip(band_3_mirror_lr, 1)
        image_mirror_lrud.append(np.dstack((band_1_mirror_lrud, band_2_mirror_lrud, band_3_mirror_lrud)))
        
        #rotate 
        band_1_rotate = rot(band_1, 45, reshape=False)
        band_2_rotate = rot(band_2, 45, reshape=False)
        band_3_rotate = rot(band_3, 45, reshape=False)
        image_rotate.append(np.dstack((band_1_rotate, band_2_rotate, band_3_rotate)))
        
        #rotate2 
        band_1_rotate_2 = rot(band_1, -45, reshape=False)
        band_2_rotate_2 = rot(band_2, -45, reshape=False)
        band_3_rotate_2 = rot(band_3, -45, reshape=False)
        image_rotate_2.append(np.dstack((band_1_rotate_2, band_2_rotate_2, band_3_rotate_2)))
        
        #rotate3 
        band_1_rotate_3 = rot(band_1, 135, reshape=False)
        band_2_rotate_3 = rot(band_2, 135, reshape=False)
        band_3_rotate_3 = rot(band_3, 135, reshape=False)
        image_rotate_3.append(np.dstack((band_1_rotate_3, band_2_rotate_3, band_3_rotate_3)))
        
        #rotate4 
        band_1_rotate_4 = rot(band_1, -135, reshape=False)
        band_2_rotate_4 = rot(band_2, -135, reshape=False)
        band_3_rotate_4 = rot(band_3, -135, reshape=False)
        image_rotate_4.append(np.dstack((band_1_rotate_4, band_2_rotate_4, band_3_rotate_4)))
        
    mirrorlr = np.array(image_mirror_lr)
    mirrorud = np.array(image_mirror_ud)
    mirrorlrud = np.array(image_mirror_lrud)
    rotated = np.array(image_rotate)
    rotated2 = np.array(image_rotate_2)
    rotated3 = np.array(image_rotate_3)
    rotated4 = np.array(image_rotate_4)
    images = np.concatenate((images, mirrorlr, mirrorud, mirrorlrud, rotated, rotated2, rotated3, rotated4))
    return images

In [12]:
X_train_more = augment(X_train) 
y_train_more = np.concatenate((y_train,y_train,y_train,y_train,y_train,y_train,y_train,y_train))

X_train_more.shape

(12832, 75, 75, 3)

In [13]:
from sklearn.model_selection import train_test_split

X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train_more, y_train_more, random_state=18, train_size=0.75)

In [14]:
#Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

#define our model
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 2
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 3
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 4
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    return gmodel


Using TensorFlow backend.


In [15]:
def getModel2():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 2
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu' ))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 3
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 4
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))
    
    gmodel.add(BatchNormalization())

    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(128))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))
    
    gmodel.add(Dense(64))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.001, decay=0.0)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    
    return gmodel

In [16]:
def getModel3():
    model = Sequential()

    model.add(Conv2D(64, kernel_size=(3,3), input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Flatten())

    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))


    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.01, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer = mypotim, metrics=['accuracy'])
    
    return model

In [17]:
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalMaxPooling2D
from keras.optimizers import SGD
from keras.layers.merge import Concatenate

def getVggModel():
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=(75,75,3), classes=1)
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu', name='fc3')(x)
    x = Dropout(0.3)(x)
    
    predictions = Dense(1, activation='sigmoid')(x)
    
    model = Model(input=base_model.input, output=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

In [18]:
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [msave,es]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=10)

In [19]:
gmodel=getModel3()

In [20]:
#gmodel.load_weights(filepath=file_path)
gmodel.fit(X_train_cv, y_train_cv,
          batch_size=100,
          epochs=100,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Train on 9624 samples, validate on 3208 samples
Epoch 1/100
9624/9624 [==============================] - 21s 2ms/step - loss: 0.4505 - acc: 0.8011 - val_loss: 1.7558 - val_acc: 0.6958
Epoch 2/100
9624/9624 [==============================] - 10s 1ms/step - loss: 0.3031 - acc: 0.8646 - val_loss: 0.3532 - val_acc: 0.8585
Epoch 3/100
9624/9624 [==============================] - 10s 1ms/step - loss: 0.2945 - acc: 0.8680 - val_loss: 0.2989 - val_acc: 0.8881
Epoch 4/100
9624/9624 [==============================] - 10s 1ms/step - loss: 0.2586 - acc: 0.8859 - val_loss: 0.3266 - val_acc: 0.8697
Epoch 5/100
9624/9624 [==============================] - 10s 1ms/step - loss: 0.2625 - acc: 0.8882 - val_loss: 0.2187 - val_acc: 0.9115
Epoch 6/100
9624/9624 [==============================] - 10s 1ms/step - loss: 0.2457 - acc: 0.8932 - val_loss: 0.3579 - val_acc: 0.8557
Epoch 7/100
9624/9624 [==============================] - 10s 1ms/step - loss: 0.2271 - acc: 0.9036 - val_loss: 0.2520 - val_acc: 0.9096


In [21]:
gmodel.load_weights(filepath=file_path)
score = gmodel.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

3208/3208 [==============================] - 1s 387us/step
Test loss: 0.155210981587
Test accuracy: 0.9463840399


In [31]:
predicted_test=gmodel.predict(X_test)

In [23]:
predicted_test = predicted_test.reshape((predicted_test.shape[0]))

In [24]:
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=predicted_test
submission.to_csv('submission_1.csv', index=False)

In [22]:
from sklearn.model_selection import StratifiedKFold

folds = 5
skf = StratifiedKFold(n_splits=folds, random_state=233)

In [23]:
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=np.zeros(len(test_df))

for train_index, test_index in skf.split(X_train_more, y_train_more):
    X_train, X_valid = X_train_more[train_index],X_train_more[test_index]
    y_train, y_valid = y_train_more[train_index],y_train_more[test_index]
    
    gmodel=getModel()
    gmodel.fit(X_train, y_train,
          batch_size=100,
          epochs=100,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)
    
    score = gmodel.evaluate(X_valid, y_valid, verbose=1)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    predicted_test=gmodel.predict(X_test)
    
    submission['is_iceberg'] += predicted_test.reshape((predicted_test.shape[0]))
        
submission['is_iceberg'] /= folds

Train on 10265 samples, validate on 2567 samples
Epoch 1/100
10265/10265 [==============================] - 7s 634us/step - loss: 0.5145 - acc: 0.7200 - val_loss: 0.3668 - val_acc: 0.8383
Epoch 2/100
10265/10265 [==============================] - 6s 562us/step - loss: 0.3503 - acc: 0.8398 - val_loss: 0.3179 - val_acc: 0.8535
Epoch 3/100
10265/10265 [==============================] - 6s 545us/step - loss: 0.2944 - acc: 0.8722 - val_loss: 0.3012 - val_acc: 0.8683
Epoch 4/100
10265/10265 [==============================] - 6s 541us/step - loss: 0.2817 - acc: 0.8806 - val_loss: 0.2616 - val_acc: 0.8894
Epoch 5/100
10265/10265 [==============================] - 6s 539us/step - loss: 0.2609 - acc: 0.8860 - val_loss: 0.2740 - val_acc: 0.8812
Epoch 6/100
10265/10265 [==============================] - 6s 543us/step - loss: 0.2568 - acc: 0.8920 - val_loss: 0.2605 - val_acc: 0.8944
Epoch 7/100
10265/10265 [==============================] - 6s 546us/step - loss: 0.2361 - acc: 0.9020 - val_loss: 0.2

10265/10265 [==============================] - 6s 540us/step - loss: 0.1973 - acc: 0.9199 - val_loss: 0.1887 - val_acc: 0.9291
Epoch 13/100
10265/10265 [==============================] - 6s 539us/step - loss: 0.1997 - acc: 0.9195 - val_loss: 0.1970 - val_acc: 0.9240
Epoch 14/100
10265/10265 [==============================] - 6s 540us/step - loss: 0.1805 - acc: 0.9262 - val_loss: 0.1790 - val_acc: 0.9299
Epoch 15/100
10265/10265 [==============================] - 6s 542us/step - loss: 0.1769 - acc: 0.9307 - val_loss: 0.1803 - val_acc: 0.9291
Epoch 16/100
10265/10265 [==============================] - 6s 540us/step - loss: 0.1705 - acc: 0.9325 - val_loss: 0.1713 - val_acc: 0.9353
Epoch 17/100
10265/10265 [==============================] - 6s 538us/step - loss: 0.1700 - acc: 0.9327 - val_loss: 0.1763 - val_acc: 0.9307
Epoch 18/100
10265/10265 [==============================] - 6s 540us/step - loss: 0.1569 - acc: 0.9363 - val_loss: 0.1600 - val_acc: 0.9353
Epoch 19/100
10265/10265 [=======

10265/10265 [==============================] - 6s 543us/step - loss: 0.3039 - acc: 0.8614 - val_loss: 0.3499 - val_acc: 0.8586
Epoch 4/100
10265/10265 [==============================] - 6s 543us/step - loss: 0.2765 - acc: 0.8795 - val_loss: 0.2692 - val_acc: 0.8761
Epoch 5/100
10265/10265 [==============================] - 6s 545us/step - loss: 0.2632 - acc: 0.8831 - val_loss: 0.2549 - val_acc: 0.8913
Epoch 6/100
10265/10265 [==============================] - 6s 539us/step - loss: 0.2599 - acc: 0.8898 - val_loss: 0.2661 - val_acc: 0.8800
Epoch 7/100
10265/10265 [==============================] - 6s 543us/step - loss: 0.2277 - acc: 0.9058 - val_loss: 0.2408 - val_acc: 0.9049
Epoch 8/100
10265/10265 [==============================] - 6s 544us/step - loss: 0.2218 - acc: 0.9069 - val_loss: 0.2429 - val_acc: 0.8991
Epoch 9/100
10265/10265 [==============================] - 6s 544us/step - loss: 0.2102 - acc: 0.9163 - val_loss: 0.2100 - val_acc: 0.9135
Epoch 10/100
10265/10265 [=============

10266/10266 [==============================] - 6s 544us/step - loss: 0.1387 - acc: 0.9450 - val_loss: 0.2379 - val_acc: 0.8959
Epoch 20/100
10266/10266 [==============================] - 6s 548us/step - loss: 0.1496 - acc: 0.9401 - val_loss: 0.2356 - val_acc: 0.9037
Epoch 21/100
10266/10266 [==============================] - 6s 548us/step - loss: 0.1389 - acc: 0.9459 - val_loss: 0.2313 - val_acc: 0.8979
Epoch 22/100
10266/10266 [==============================] - 6s 545us/step - loss: 0.1434 - acc: 0.9417 - val_loss: 0.2563 - val_acc: 0.8979
Epoch 23/100
10266/10266 [==============================] - 6s 544us/step - loss: 0.1258 - acc: 0.9493 - val_loss: 0.2412 - val_acc: 0.9026
Epoch 24/100
10266/10266 [==============================] - 6s 544us/step - loss: 0.1148 - acc: 0.9552 - val_loss: 0.2218 - val_acc: 0.9111
Epoch 25/100
10266/10266 [==============================] - 6s 545us/step - loss: 0.1114 - acc: 0.9568 - val_loss: 0.2883 - val_acc: 0.8967
Epoch 26/100
10266/10266 [=======

In [24]:
submission.to_csv('submission3.csv', index=False)